In [16]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [17]:
#https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere
#Create n evenly distributed points on a sphere

num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))

x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")

fig.show()
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_ibmq_ourense.csv",target_points)

In [18]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])
    
circuit.ry(1.107149, qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[2])
circuit.ry(0.729728, qreg_q[2])
circuit.cx(qreg_q[2], qreg_q[1])
circuit.ry(0.463648, qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[2])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.cx(qreg_q[2], qreg_q[0])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[0])
circuit.rz(-phi_param, qreg_q[1])
circuit.ry(-theta_param, qreg_q[1])

circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])

In [19]:
IBMQ.save_account('3e51c922457f0cf5a1aca173ef4ec0e143f8bc67ca155d51f79c82014610fe57f85c943c4d6e34e6494daaf860b248e815661e379eef57f160258bd1ecf7321b')

N_shots=8192
provider = IBMQ.load_account()

#backend = least_busy(provider.backends(simulator=False))
#print(backend)
backend = provider.backends.ibmq_ourense

configrc.store_credentials:WARNING:2020-12-05 16:22:28,861: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2020-12-05 16:22:29,433: Credentials are already in use. The existing account in the session will be replaced.


In [20]:
runningjobs=[]
job_ids=[]
index=0

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend, optimization_level=0), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_ourense.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend, optimization_level=0), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_ourense.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    index=index+1

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='89a0160e-46b3-4298-8460-e321cfcb6db1', job_id='5fcba5bbeaa71d001a289326', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5486, '0x1': 1030, '0x2': 1291, '0x3': 385}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=2.5416018461576297, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5238, '0x1': 1142, '0x2': 1422, '0x3': 390}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.3416202312933034, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]]

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='924983c6-10d9-4072-a64f-59af9c4a52c5', job_id='5fcba5c18fd6b8001988f1d2', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5279, '0x1': 1111, '0x2': 1335, '0x3': 467}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.5075750318473027, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5164, '0x1': 1118, '0x2': 1473, '0x3': 437}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=2.4491860705728037, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]]

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='e4515cde-21bb-4d98-8432-ac1a14e16edf', job_id='5fcba5c5eaa71d001a289327', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5198, '0x1': 1084, '0x2': 1344, '0x3': 566}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.6151408711267798, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5184, '0x1': 1162, '0x2': 1355, '0x3': 491}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.4151592562625446, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]]

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='73fb17df-e891-4bc5-8683-75f76fa2c2c0', job_id='5fcba5ca21d781001a24d3bb', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4488, '0x1': 1478, '0x2': 1796, '0x3': 430}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=2.722706710406314, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 5007, '0x1': 1159, '0x2': 1415, '0x3': 611}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.522725095541965, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], 

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='0fbd3517-584e-4860-b54b-07f2bd35f416', job_id='5fcba5ce542cea0018e805dd', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4704, '0x1': 1249, '0x2': 1679, '0x3': 560}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.688679896095941, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4508, '0x1': 1502, '0x2': 1678, '0x3': 504}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=2.630290934821385, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], 

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='e402ad0a-14c6-4acd-9be9-e63e74806be8', job_id='5fcba9838fd6b8001988f1f3', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4830, '0x1': 1091, '0x2': 1655, '0x3': 616}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.7962457353755887, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4647, '0x1': 1269, '0x2': 1703, '0x3': 573}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.5962641205112398, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]]

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='32e62d75-5d7f-4230-bf14-f0eaec707ecf', job_id='5fcbaa3b542cea0018e80604', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4482, '0x1': 1418, '0x2': 1730, '0x3': 562}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=2.9038115746547817, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4778, '0x1': 1090, '0x2': 1646, '0x3': 678}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.7038299597908875, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]]

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='0d27d77a-adc9-4b2f-a4e8-656c8d5c8b7f', job_id='5fcbaaf21f6c7f001bfd8979', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4310, '0x1': 1394, '0x2': 1930, '0x3': 558}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.8697847603446363, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4437, '0x1': 1485, '0x2': 1718, '0x3': 552}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=2.8113957990700804, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]]

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='32b56f2a-7708-4b79-b86e-7d900814fe36', job_id='5fcbabab1115b9001b2aac3b', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4696, '0x1': 1154, '0x2': 1779, '0x3': 563}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.977350599624284, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4370, '0x1': 1360, '0x2': 1928, '0x3': 534}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.777368984759935, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], 

Maximum number of jobs reached. Waiting...
The job finished with result Result(backend_name='ibmq_ourense', backend_version='1.3.3', qobj_id='138d402d-6627-4851-8282-512d225f375b', job_id='5fcbac62c6d8d2001971f305', success=True, results=[ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4487, '0x1': 1305, '0x2': 1889, '0x3': 511}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=3.084916438903477, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]]), memory=False), ExperimentResult(shots=8192, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 4789, '0x1': 1067, '0x2': 1754, '0x3': 582}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=1.8849348240395827, memory_slots=3, n_qubits=5, name='circuit8', qreg_sizes=[['q', 5]],

In [21]:
print(index)

14


In [25]:
read_job_ids = open("job_ids_ourense.txt").read().splitlines()
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    marg_prob0_qubit0=job["000"]/N_shots+job["010"]/N_shots #calculate the marginal distribution
    marg_prob0_qubit1=job["000"]/N_shots+job["001"]/N_shots #calculate the marginal distribution
    marg_prob1_qubit0=1-marg_prob0_qubit0
    marg_prob1_qubit1=1-marg_prob0_qubit1
    results_probabilities.append([marg_prob0_qubit0,marg_prob1_qubit0, marg_prob0_qubit1, marg_prob1_qubit1])
    
results_probabilities=np.array(results_probabilities)

In [26]:
for (point,probabilities) in zip(target_points,results_probabilities):
    with open('results_ibm_ourense.txt', 'a') as file:
        file.write(str(point[0])+"\t"+str(point[1])+"\t"+str(probabilities[0])+"\t"+str(probabilities[1])+"\t"+
                   str(probabilities[2])+"\t"+str(probabilities[3])+"\n")

In [27]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [28]:
data_sheet.head()

,θ,ϕ,prob_0_qubit_0,prob_1_qubit_0,prob_0_qubit_1,prob_1_qubit_1
0,0.044725,5.083204,0.827271,0.172729,0.795410,0.204590
1,0.077479,2.683240,0.812988,0.187012,0.778809,0.221191
2,0.100042,0.283277,0.812866,0.187134,0.795898,0.204102
3,0.118391,4.166499,0.818604,0.181396,0.784668,0.215332
4,0.134265,1.766536,0.808228,0.191772,0.798340,0.201660


In [29]:
# save to excel, optional
data_sheet.to_excel("data_sheet_ibm_ourense.xlsx")

In [30]:
data_sheet=pd.read_excel("data_sheet_ibm_ourense.xlsx")

In [31]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  1000
Average fidelity:  0.77528466796875
Standard deviation:  0.027397981155289058


In [32]:
#FIDELITY OF SECOND COPY
# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_1/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_1))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_1))

Number of points:  1000
Average fidelity:  0.7176956787109375
Standard deviation:  0.03467978203264979


In [33]:
#CHECK SYMMETRY, I.E. PLOT fidelity of qubit 0/fidelity of qubit 2

df = px.data.iris()
ratio=data_sheet.prob_0_qubit_0/data_sheet.prob_0_qubit_1
fig = px.scatter_3d(df, x, y, z, color=ratio, template="simple_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(ratio))
print("Standard deviation: ", np.std(ratio))

Number of points:  1000
Average fidelity:  1.0812980206641196
Standard deviation:  0.032221664489662634
